In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import glob
import os

all_harga_pangan = glob.glob("Harga Bahan Pangan/train/*")

In [30]:
provinces = [
    "Aceh", "Sumatera Utara", "Sumatera Barat", "Riau", "Jambi", "Sumatera Selatan", "Bengkulu",
    "Lampung", "Kepulauan Bangka Belitung", "Kepulauan Riau", "DKI Jakarta", "Jawa Barat", "Jawa Tengah",
    "DI Yogyakarta", "Jawa Timur", "Banten", "Bali", "Nusa Tenggara Barat", "Nusa Tenggara Timur",
    "Kalimantan Barat", "Kalimantan Tengah", "Kalimantan Selatan", "Kalimantan Timur", "Kalimantan Utara",
    "Sulawesi Utara", "Sulawesi Tengah", "Sulawesi Selatan", "Sulawesi Tenggara", "Gorontalo",
    "Sulawesi Barat", "Maluku", "Maluku Utara", "Papua", "Papua Barat"
]

In [3]:
commodity_dict = {}

for file in all_harga_pangan:
    df = pd.read_csv(file)
    
    filename = os.path.basename(os.path.splitext(file)[0])
    first_word = " ".join(filename.split()[:1]).lower()
    first_two_words = " ".join(filename.split()[:2]).lower()  # Extract first two words

    trend_file = f"Google Trend Merged/{first_two_words}.csv"

    if first_word == "beras" or first_word == "gula":
        trend_df = pd.read_csv(f"Google Trend Merged/{first_word}.csv")
    else:
        trend_df = pd.read_csv(f"Google Trend Merged/{first_two_words}.csv")

    
    commodity_dict[filename] = []
    
    for col in df.columns[1:]:
        # print(filename)
        # print(col)

        if col not in trend_df:
            trend_df[col] = pd.Series(np.zeros(df.shape[0]))

        split_df = df[["Date", col]].ffill().bfill().copy()
        split_df["trend"] = trend_df[col].fillna(0)
    

        commodity_dict[filename].append(split_df)



In [5]:
commodity_price_df = pd.read_csv("global_commodity_price_merged.csv")

def append_feat(a, b):
    if commodity.startswith(a):
        lst = commodity_dict[commodity]
        new_lst = []
        
        for df in lst:
            new_df = pd.merge(right=df, left=commodity_price_df[["Date", f"Price_{b}", f"Change_{b}"]], how="outer", on="Date")
            new_df[f"Price_{b}"] = new_df[f"Price_{b}"].ffill().bfill()
            new_df[f"Change_{b}"] = new_df[f"Change_{b}"].fillna(0)
            new_lst.append(new_df)
        commodity_dict[commodity] = new_lst    

for commodity in commodity_dict:
    append_feat("Minyak Goreng", "Palm Oil Futures Historical Data")
    append_feat("Tepung Terigu", "US Wheat Futures Historical Data")
    append_feat("Gula", "US Sugar 11 Futures Historical Data")

In [24]:
from sklearn.ensemble import RandomForestRegressor

In [38]:
train_num = int(1004 * 0.8)
test_num = 1004 - train_num

for commodity in commodity_dict:
    for df in commodity_dict[commodity]:
        train_df = df[:train_num]
        test_df = df[train_num:]

        # print("Train")
        # print(train_df)
        # print("Test")
        # print(test_df)

        province_columns = [col for col in train_df.columns if col in provinces]

        X_train = train_df.drop(columns=province_columns)
        y_train = train_df[province_columns]
        X_test = test_df.drop(columns=province_columns)
        y_test = test_df[province_columns]

        model = RandomForestRegressor(n_estimators=100, random_state=10)
        model.fit(X_train, y_train)
        print(model.score(X_test, y_test))

        

    

ValueError: could not convert string to float: '2022-01-01'